In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Folder with the Furman neighborhood Excel files
FOLDER = Path("/Users/victoriadynak/Desktop/nyc_datacleaning/neighborhoods_furman")  # <-- adjust if needed
assert FOLDER.exists(), f"Folder not found: {FOLDER}"

In [2]:
# Cell: Parser for one Furman workbook (sheet "* Data")
def to_numeric_clean(x):
    if isinstance(x, str):
        s = x.strip()
        if s.endswith("%"): s = s[:-1]
        s = s.replace("$","").replace(",","")
        try:
            return float(s)
        except:
            return np.nan
    return pd.to_numeric(x, errors="coerce")

def parse_furman_profile_one_workbook(path: Path) -> pd.DataFrame:
    xls = pd.ExcelFile(path)
    data_sheets = [s for s in xls.sheet_names if "Data" in s]
    if not data_sheets:
        raise ValueError(f"No sheet with 'Data' in {path.name}")
    raw = pd.read_excel(xls, sheet_name=data_sheets[0], header=None)

    # locate header row
    hdr_mask = raw.apply(lambda r: r.astype(str).str.contains("Community District", case=False, na=False).any(), axis=1)
    hdr_idx = hdr_mask.idxmax()
    header = list(raw.iloc[hdr_idx])
    df = raw.iloc[hdr_idx+1:].copy()
    df.columns = header
    df = df.dropna(how="all").dropna(how="all", axis=1)

    df = df.rename(columns={
        "Community District":"community_district",
        "Name":"name",
        "Indicator Category":"indicator_category",
        "Indicator":"indicator",
        "Indicator Description":"indicator_description",
    })

    year_cols = [c for c in df.columns if str(c).isdigit() and len(str(c))==4]
    id_vars = ["community_district","name","indicator_category","indicator","indicator_description"]
    id_vars = [c for c in id_vars if c in df.columns]

    long_df = df.melt(id_vars=id_vars, value_vars=year_cols, var_name="year", value_name="value")
    long_df["value"] = long_df["value"].apply(to_numeric_clean)
    long_df["year"]  = pd.to_numeric(long_df["year"], errors="coerce").astype("Int64")
    long_df = long_df.dropna(subset=["year"])
    long_df["community_district"] = long_df["community_district"].astype(str).str.strip()
    long_df["name"] = long_df["name"].astype(str).str.strip()
    long_df["indicator"] = long_df["indicator"].astype(str).str.strip()
    return long_df


In [3]:
# Cell: Build wide_all from all workbooks; compute quality_score
files = sorted([p for p in FOLDER.glob("*.xlsx")] + [p for p in FOLDER.glob("*.xls")])
assert files, f"No Excel files found in {FOLDER}"

frames = []
for p in files:
    try:
        frames.append(parse_furman_profile_one_workbook(p))
    except Exception as e:
        print(f"Skip {p.name}: {e}")

tidy_all = pd.concat(frames, ignore_index=True)
# keep key years (you can change)
KEEP_YEARS = {2010, 2019, 2021, 2022}
tidy_all = tidy_all[tidy_all["year"].isin(KEEP_YEARS)]

# pivot to wide (one row per CD-year, columns per indicator)
wide_all = (tidy_all
    .pivot_table(index=["community_district","name","year"],
                 columns="indicator", values="value", aggfunc="first")
    .reset_index()
)

# define negative indicators (higher = worse) so we flip after z-scaling
NEGATIVE = {
    "Poverty rate",
    "Poverty rate, population aged 65+",
    "Poverty rate, population under 18 years old",
    "Unemployment rate",
    "Gross rent as a share of household income (rent burden)",
    "Moderately rent-burdened households",
    "Moderately rent-burdened households, low income",
    "Moderately rent-burdened households, moderate income",
    "Severely rent-burdened households",
    "Severely rent-burdened households, low income",
    "Severely rent-burdened households, moderate income",
    "Serious housing code violations (per 1,000 privately owned rental units)",
    "Total housing code violations (per 1,000 privately owned rental units)",
    "Serious crime rate (per 1,000 residents)",
    "Serious crime rate, violent (per 1,000 residents)",
    "Serious crime rate, property (per 1,000 residents)",
    "Notices of foreclosure rate (per 1,000 1-4 family and condo properties)",
    "Pre-foreclosure notice rate (per 1,000 1-4 family and condo properties)",
    "Severe crowding rate (% of renter households)",
    "Mean travel time to work (minutes)",
}

# z-score across citywide for each year (so comparisons are city-relative)
id_cols = ["community_district","name","year"]
value_cols = [c for c in wide_all.columns if c not in id_cols]
X_list = []
for y, block in wide_all.groupby("year"):
    X = block[value_cols].apply(pd.to_numeric, errors="coerce")
    scaler = StandardScaler()
    Xz = pd.DataFrame(scaler.fit_transform(X), columns=value_cols, index=block.index)
    # flip negatives
    for col in Xz.columns:
        if col in NEGATIVE:
            Xz[col] = -Xz[col]
    # composite = mean of available signals; require ≥3 present
    enough = Xz.notna().sum(axis=1) >= 3
    qs = Xz.mean(axis=1, skipna=True)
    block = block.copy()
    block["quality_score"] = np.where(enough, qs, np.nan)
    X_list.append(block)

wide_all = pd.concat(X_list).sort_values(id_cols).reset_index(drop=True)


/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/v

In [4]:
# Cell: Top-3 drivers (uses the same Xz logic as above, per year)
def compute_Xz_per_year(wide, value_cols, negative_set):
    out = {}
    for y, block in wide.groupby("year"):
        X = block[value_cols].apply(pd.to_numeric, errors="coerce")
        scaler = StandardScaler()
        Xz = pd.DataFrame(scaler.fit_transform(X), columns=value_cols, index=block.index)
        for col in Xz.columns:
            if col in negative_set:
                Xz[col] = -Xz[col]
        out[y] = Xz
    return out

Xz_by_year = compute_Xz_per_year(wide_all, value_cols, NEGATIVE)

def top_k_drivers_for_row(row, k=3):
    y = int(row["year"])
    idx = row.name
    z = Xz_by_year[y].loc[idx].dropna()
    if z.empty: return ""
    top = z.reindex(z.abs().sort_values(ascending=False).index)[:k]
    nice_names = {
        "Median household income (2024$)": "Income",
        "Homeownership rate": "Homeownership",
        "Labor force participation rate": "Labor force",
        "Students performing at grade level in English language arts, 4th grade": "ELA prof.",
        "Students performing at grade level in math, 4th grade": "Math prof.",
        "Racial diversity index": "Racial diversity",
        "Poverty rate": "Poverty",
        "Unemployment rate": "Unemployment",
        "Gross rent as a share of household income (rent burden)": "Rent burden",
        "Serious housing code violations (per 1,000 privately owned rental units)": "Serious code viol.",
        "Total housing code violations (per 1,000 privately owned rental units)": "All code viol.",
        "Serious crime rate (per 1,000 residents)": "Crime (serious)",
        "Serious crime rate, violent (per 1,000 residents)": "Crime (violent)",
        "Serious crime rate, property (per 1,000 residents)": "Crime (property)",
        "Notices of foreclosure rate (per 1,000 1-4 family and condo properties)": "Foreclosure rate",
        "Pre-foreclosure notice rate (per 1,000 1-4 family and condo properties)": "Pre-foreclosure rate",
        "Severe crowding rate (% of renter households)": "Severe crowding",
        "Mean travel time to work (minutes)": "Commute time",
    }
    return ", ".join([f"{nice_names.get(c,c)} ({v:+.2f})" for c, v in top.items()])

wide_all["top3_drivers"] = wide_all.apply(top_k_drivers_for_row, axis=1)


/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/victoriadynak/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/v

In [ ]:
# Cell: Yearly -> monthly interpolation for ALL numeric columns
def interpolate_all_numeric(df_cd):
    # index by 1/1/year
    dt_index = pd.to_datetime(df_cd['year'].astype(int).astype(str) + "-01-01")
    df = df_cd.set_index(dt_index)
    # numeric-only
    num_cols = df.select_dtypes(include=[np.number]).columns
    df = df[num_cols]
    # monthly range & interpolation
    rng = pd.date_range(df.index.min(), df.index.max(), freq='MS')
    df = df.reindex(rng)
    df = df.interpolate(method='time')
    df['month'] = df.index
    return df.reset_index(drop=True)

monthly_rows = []
for (cd, nm), g in wide_all.groupby(["community_district","name"], dropna=False):
    m = interpolate_all_numeric(g)
    m["community_district"] = cd
    m["name"] = nm
    monthly_rows.append(m)

furman_monthly = pd.concat(monthly_rows, ignore_index=True).sort_values(["community_district","month"])

# future 6-month label (what you'll predict)
assert "quality_score" in furman_monthly.columns, "quality_score missing—check earlier cells."
furman_monthly["quality_score_t_plus_6m"] = (
    furman_monthly.groupby("community_district")["quality_score"].shift(-6)
)

# user-facing fields (optional)
furman_monthly["quality_percentile_month"] = (
    furman_monthly.groupby("month")["quality_score"].rank(pct=True) * 100
)

qmin, qmax = furman_monthly["quality_score"].min(), furman_monthly["quality_score"].max()
furman_monthly["quality_index_0_100"] = 100 * (furman_monthly["quality_score"] - qmin) / (qmax - qmin)

def to_grade(p):
    if pd.isna(p): return None
    if p >= 90: return "A+"
    if p >= 80: return "A"
    if p >= 70: return "B"
    if p >= 60: return "C"
    if p >= 40: return "D"
    return "F"

furman_monthly["quality_grade"] = furman_monthly["quality_percentile_month"].apply(to_grade)

# carry yearly drivers down to months
drivers_yearly = (wide_all[["community_district","year","top3_drivers"]]
                  .assign(month=lambda d: pd.to_datetime(d["year"].astype(int).astype(str)+"-01-01")))
furman_monthly = furman_monthly.merge(
    drivers_yearly[["community_district","month","top3_drivers"]],
    on=["community_district","month"], how="left"
)
furman_monthly["year_tmp"] = furman_monthly["month"].dt.year
furman_monthly["top3_drivers"] = (furman_monthly
    .sort_values(["community_district","month"])
    .groupby(["community_district","year_tmp"])["top3_drivers"]
    .ffill()
)
furman_monthly = furman_monthly.drop(columns=["year_tmp"])

# optional: save
furman_monthly.head()


indicator,year,Affordable Units Completed (<=50% AMI),"Affordable Units Completed per capita (<=50% AMI, adjusted by 100,000 people)",Born in New York State,Car-free commute (% of commuters),Change in rent (2022-2023),Disabled population,FHA/VA-backed home purchase loans (% of home purchase loans),Foreign-born population,Higher-cost home purchase loans (% of home purchase loans),...,Units issued new certificates of occupancy,quality_score,month,community_district,name,quality_score_t_plus_6m,quality_percentile_month,quality_index_0_100,quality_grade,top3_drivers
0,2010.0,NaN,NaN,52.000000,85.200000,NaN,5.000000,17.300000,25.800000,0.400000,...,2496.000000,0.043757,2010-01-01,BK 01,Greenpoint/Williamsburg,0.055361,67.796610,53.428668,C,Units issued new certificates of occupancy (+4...
1,2010.08488,NaN,NaN,51.956617,85.205659,NaN,4.999057,17.140615,25.784910,0.403772,...,2491.180712,0.045744,2010-02-01,BK 01,Greenpoint/Williamsburg,0.057348,69.491525,53.628461,C,Units issued new certificates of occupancy (+4...
2,2010.161545,NaN,NaN,51.917432,85.210770,NaN,4.998205,16.996653,25.771281,0.407180,...,2486.827807,0.047540,2010-03-01,BK 01,Greenpoint/Williamsburg,0.059335,69.491525,53.808920,C,Units issued new certificates of occupancy (+4...
3,2010.246425,NaN,NaN,51.874049,85.216428,NaN,4.997262,16.837268,25.756191,0.410952,...,2482.008518,0.049527,2010-04-01,BK 01,Greenpoint/Williamsburg,0.061259,69.491525,54.008713,C,Units issued new certificates of occupancy (+4...
4,2010.328567,NaN,NaN,51.832066,85.221904,NaN,4.996349,16.683024,25.741588,0.414603,...,2477.344691,0.051450,2010-05-01,BK 01,Greenpoint/Williamsburg,0.063246,69.491525,54.202061,C,Units issued new certificates of occupancy (+4...


In [7]:
if "month" not in furman_monthly.columns or not np.issubdtype(furman_monthly["month"].dtype, np.datetime64):
    # fallback: derive month from a decimal year column if needed
    if "year" in furman_monthly.columns:
        y = np.floor(furman_monthly["year"].astype(float)).astype(int)
        frac = furman_monthly["year"].astype(float) - y
        # convert fractional year to month number (1..12)
        m = (frac * 12).round().astype(int) + 1
        m = m.clip(1, 12)
        furman_monthly["month"] = pd.to_datetime(dict(year=y, month=m, day=1))
    else:
        raise ValueError("No 'month' or 'year' available to derive a monthly date.")

# force to month-start (safety)
furman_monthly["month"] = pd.to_datetime(furman_monthly["month"]).dt.to_period("M").dt.to_timestamp()

# --- 2) Add a clean integer year (optional, useful for grouping/QA) ---
furman_monthly["year_int"] = furman_monthly["month"].dt.year

# --- 3) Drop the decimal 'year' column to avoid confusion ---
if "year" in furman_monthly.columns:
    # keep it only if you really need it for auditing; otherwise drop:
    furman_monthly = furman_monthly.drop(columns=["year"])

# --- 4) Recompute label (in case ordering changed) ---
if "quality_score" not in furman_monthly.columns:
    raise KeyError("`quality_score` missing—create it on wide_all before interpolation.")
furman_monthly = furman_monthly.sort_values(["community_district","month"])
furman_monthly["quality_score_t_plus_6m"] = (
    furman_monthly.groupby("community_district")["quality_score"].shift(-6)
)

# --- 5) Final column order (put keys up front) ---
key_cols = ["community_district","name","month","year_int","quality_score","quality_score_t_plus_6m"]
other_cols = [c for c in furman_monthly.columns if c not in key_cols]
furman_monthly = furman_monthly[key_cols + other_cols]

furman_monthly.head()
furman_monthly.to_csv("furman_monthly.csv", index=False)